In [ ]:
import numpy as np
import pandas as pd
import scipy

from IPython.display import clear_output

import sys

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../general/src/')

from sklearn.metrics import auc
from scipy.stats import fisher_exact
pd.options.display.precision = 3
pd.options.display.expand_frame_repr = False
pd.options.display.max_columns = 20

In [ ]:
import cv2
import pytesseract

import re
from pathlib import Path
import requests
import random
import time
import scipy
from bs4 import BeautifulSoup

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
concat_array = []
files = np.array(os.listdir('../data/'))
relevant_files = files[['sem_images_231227_batch_count' in x for x in files]]
for file in relevant_files:
    concat_array.append(pd.read_csv('../data/'+file))
meta_df = pd.concat(concat_array)

In [ ]:
meta_df.to_csv('../data/sem_images_231227_batch.csv', index=False)

In [ ]:
remove = []
for filename in meta_df['filename'].values:
    if 'lrg' in filename:
        remove.append(filename.replace('_lrg', ''))

In [ ]:
meta_df = meta_df[~meta_df['filename'].isin(remove)].copy()

In [ ]:
def get_journal(x):
    if 's1369' in x:
        return 'mattod'
    else:
        return x.split('.')[2]

In [ ]:
def get_year(x):
    if 's1369' in x:
        return np.nan
    else:
        return x.split('.')[3]

In [ ]:
meta_df['journal'] = meta_df['doi'].apply(lambda x: get_journal(x))
meta_df['year'] = meta_df['doi'].apply(lambda x: get_year(x))

In [ ]:
journal_names = list(meta_df['journal'].value_counts().index)
journal_names.reverse() #inplace

In [ ]:
meta_df = meta_df.sort_values(['journal_categorical', 'year']).copy()

In [ ]:
meta_df.groupby('journal').nunique().sort_values('filename')

In [ ]:
%%time
min_confidence = 50

for journal_name in ['pone']:
    result_df_array = []
    meta_df_slice = meta_df[meta_df['journal'] == journal_name].reset_index(drop=True)
    for index, row in meta_df_slice.iterrows():

        filename = row['filename']
        img_path = '../data/img/' + filename
        try:
            scale = 1.0
            img = cv2.imread(img_path)

            tesseract_results = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)

            text_array = np.array(tesseract_results['text'])\
            [np.array([float(x) for x in tesseract_results['conf']]) > min_confidence]
            result_df_array.append(pd.DataFrame({'filename':filename, 'text':text_array, 'scale':scale}))

            for scale in [1.5, 2.0]:
                img2 = cv2.resize(img, (int(img.shape[0]*scale), int(img.shape[1]*scale)))
                tesseract_results = pytesseract.image_to_data(img2, output_type=pytesseract.Output.DICT)

                text_array = np.array(tesseract_results['text'])\
                [np.array([float(x) for x in tesseract_results['conf']]) > min_confidence]
                result_df_array.append(pd.DataFrame({'filename':filename, 'text':text_array, 'scale':scale}))
        except:
            print('Error processing image.')

        if (index % 1000 == 0) & (index != 0):
            text_df = pd.concat(result_df_array)
            text_df.to_csv('../data/tesseract_ocr_sem_batch_231204_' + journal_name + '_' + str(index) + '.csv', index=False)
            result_df_array = []

        print(str(index + 1) + ' of ' + str(len(meta_df_slice)) + ' images processed from journal ' + journal_name)
        if index % 10 == 0:
            clear_output()
            
    text_df = pd.concat(result_df_array)
    text_df.to_csv('../data/tesseract_ocr_sem_batch_231204_' + journal_name + '_' + str(index) + '.csv', index=False)

In [ ]:
journals = ['msea']
files = np.array(os.listdir('../data/'))

for journal in journals:
    print(journal)
    concat_array = []
    relevant_files = files[['tesseract_ocr_sem_batch_231217_' + journal in x for x in files]]
    for file in relevant_files:
        concat_array.append(pd.read_csv('../data/'+file))
    text_df = pd.concat(concat_array)
    text_df.to_csv('../data/tesseract_ocr_sem_batch_231217_' + journal + '.csv', index=False)